# News

In this assignment, you will use the `news.csv` dataset to train a neural network that predicts a category of an article (science, politics or technology) by a given title.
    
***Notes:***

- Some parts of the code are already provided. **Do not modify the existing code.**
- Write your solution only in the sections marked with `### YOUR SOLUTION`.
- You can verify automatically graded tasks using the cell labeled `### TEST` after each function.

***IMPORTANT NOTE:***
- Name your Jupyter Notebook as `news_{index}.ipynb`.
- For example, if your index is 123456, you should name your notebook as `news_12346.ipynb`.

In [1]:
import hashlib
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
def hash_data_frame(df):
    df_sorted = df.sort_index(axis=1).sort_values(by=list(df.columns))
    return hashlib.sha256(pd.util.hash_pandas_object(df_sorted, index=True).values).hexdigest()

In [3]:
def hash_series(series):
    series_str = ",".join(map(str, series.values))
    return hashlib.sha256(series_str.encode()).hexdigest()

In [4]:
def hash_vocab(vocab):
    items = sorted(vocab.items(), key=lambda x: x[1])  
    vocab_str = "\n".join(f"{k}:{v}" for k, v in items)
    return hashlib.sha256(vocab_str.encode("utf-8")).hexdigest()

In [5]:
def hash_sequences(sequences):
    flat = []
    for seq in sequences:
        flat.extend(seq)
        flat.append(-1)  # separator to preserve sequence boundaries
    return hashlib.sha256(str(flat).encode("utf-8")).hexdigest()

In [6]:
def hash_tensor(tensor):
    return hashlib.sha256(tensor.cpu().numpy().tobytes()).hexdigest()

In [7]:
def hash_number(x):
    return hashlib.sha256(str(x).encode("utf-8")).hexdigest()

In [8]:
def hash_loss_fn(x):
    return hashlib.sha256(type(x).__name__.encode()).hexdigest()

In [9]:
df = pd.read_csv("news.csv")

df.sample()

,title,category
18,Kid Accidentally Discovered 65 Million Years O...,science


In [10]:
### AUTOMATICALLY GRADED TASK
def preprocess_text(df):
    """
    Preprocesses the 'title' column by converting all text to lowercase and removing the news 
    containing more than 50 words and ensure that the resulting DataFrame has a sequential index. 

    Returns: 
        pandas DataFrame.
    """

    ### BEGIN SOLUTION
    df["title"] = df["title"].str.lower()
    df = df[df["title"].str.split().apply(len) <= 50]
    df = df.reset_index(drop=True)
    return df
    ### END SOLUTION

In [11]:
df = preprocess_text(df)

In [13]:
### TEST
_df = pd.read_csv("news.csv")
_df = preprocess_text(_df)
assert hash_data_frame(_df) == "e703c2a4195678751ef2654c16f87274f58e93445732b3d0763bc53c7d1d6096"

In [14]:
### AUTOMATICALLY GRADED TASK
def partition_dataset(df):
    """
    Split the dataset into features (X) and target (y), where the feature used is `title` and the target is `category`.
    First divide training and testing datasets with 80:20 ratio. Afterwards, the validation dataset is obtaing from the training 
    dataset using an 80:20 ratio.

    If you think encoding is necessary use df[column].astype("category").cat.codes

    Use `random_state=42` to ensure reproducibility.
    
    Return:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame] in the following order: 
        train_X, val_X, test_X, train_y, val_y, test_y
    """

    ### BEGIN SOLUTION
    train_X, test_X, train_y, test_y = train_test_split(df["title"], df["category"].astype("category").cat.codes, test_size=0.2, random_state=42)
    train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.2, random_state=42)
    return train_X, val_X, test_X, train_y, val_y, test_y
    ### END SOLUTION

In [15]:
train_X, val_X, test_X, train_y, val_y, test_y = partition_dataset(df)

In [16]:
## TEST
_df = pd.read_csv("news.csv")
_train_X, _val_X, _test_X, _train_y, _val_y, _test_y = partition_dataset(_df)
assert hash_series(_train_X) == "59c8ed9a0b2917930e3850c056c94d2e44b8a52b4836e366347ac7c9a6ca43a6"
assert hash_series(_val_X) == "279ab9652b14dece44420a1cb618558792cfbfaad3b8fe9958f69e8aafaf00f3"
assert hash_series(_test_X) == "7dbf8caf0a6fbb7adefede4a30a7e8aa286b1a6254985ac135b537712f6e5108"
assert hash_series(_train_y) == "74f85d07212343107aab5170dfd629a1419839c36b839fad04c44447d255526b"
assert hash_series(_val_y) == "f268b980b61a2ad78dd31de78dd6cbb37864a2d5ed226e9b4144e6b8ce77dd9b"
assert hash_series(_test_y) == "933511ffa45cbf19573eba1854f0ac013d638150046f9ce21288657062c4fc47"

In [17]:
### AUTOMATICALLY GRADED TASK
def build_vocab(texts):
    """
    Builds a vocabulary dictionary from a list of text strings.

    The function should count word frequencies across all input texts and
    assign an integer index to each word. The vocabulary must include the
    special tokens "<PAD>" with index 0 and "<OOV>" with index 1. The remaining
    words should be added in descending order of frequency, such that all words
    from the texts are included.

    Parameters:
        texts (list of str): List of preprocessed text strings

    Returns:
        dict: A dictionary mapping words to integer indices
    """

    ### BEGIN SOLUTION
    counter = Counter()

    for sentence in texts:
        counter.update(sentence.split())

    # Special tokens
    vocab = {
        "<PAD>": 0,
        "<OOV>": 1
    }

    for idx, (word, _) in enumerate(
        counter.most_common(), start=2
    ):
        vocab[word] = idx

    return vocab
    ### END SOLUTION

In [18]:
vocab = build_vocab(train_X.values)

In [19]:
## TEST
assert hash_vocab(vocab) == "862a1637b460ef2f0844d5950138b65d518eedc79dced1efa06106fc6e2d4576"

In [20]:
### AUTOMATICALLY GRADED TASK
def texts_to_sequences(texts, vocab):
    """
    Converts a list of text strings into sequences of integer indices using a
    given vocabulary.

    Each word in a text should be replaced by its corresponding index from the
    vocabulary. Words that are not present in the vocabulary must be mapped to
    the "<OOV>" token.

    Parameters:
    texts (List[str]): List of preprocessed text strings
    vocab (dict): Vocabulary mapping words to integer indices

    Returns:
        List[List[int]]: List of integer sequences corresponding to the input texts
    """  
    
    ### BEGIN SOLUTION
    sequences = []

    for sentence in texts:
        seq = [
            vocab.get(word, vocab["<OOV>"])
            for word in sentence.split()
        ]
        sequences.append(seq)

    return sequences
    ### END SOLUTION

In [21]:
train_X = texts_to_sequences(train_X.values, vocab)
val_X   = texts_to_sequences(val_X.values, vocab)
test_X  = texts_to_sequences(test_X.values, vocab)

In [22]:
##TEST
assert hash_sequences(train_X) == "294f732c5d879b074ac644137de52ad86e251f6878350c4784f5e91e0a517905"
assert hash_sequences(val_X) == "335c1de506a96b5c0504aca00af07c4c8963019d1d7d4a79c6ca454f3370e018"
assert hash_sequences(test_X) == "24dc4762638be599bd79a4b8fb6b801cebdcb60d8a368b404b8ab541e67cb46a"

In [23]:
### AUTOMATICALLY GRADED TASK
def pad(sequences, pad_value=0):
    """
    Pads a list of integer sequences so that all sequences have the same length.

    Shorter sequences should be padded with the specified padding value until
    they match the length of the longest sequence. The output should be a
    tensor suitable for batch processing in a neural network.

    Parameters:
        sequences (List[List[int]]): List of integer sequences
        pad_value (int): Value used for padding (default is 0)

    Returns:
        torch.Tensor: Padded tensor of shape (batch_size, max_sequence_length)
    """
    
    ### BEGIN SOLUTION
    return pad_sequence(
        [torch.tensor(seq, dtype=torch.long) for seq in sequences],
        batch_first=True,
        padding_value=pad_value
    )
    ### END SOLUTION

In [24]:
train_X = pad(train_X, pad_value=vocab["<PAD>"])
val_X   = pad(val_X,pad_value=vocab["<PAD>"])
test_X  = pad(test_X, pad_value=vocab["<PAD>"])

In [25]:
##TEST 
assert hash_tensor(train_X) == "d0745d7e324a9a9615ead3813de202dcbe8364093c6f0049cc1a0eafc9842461"
assert hash_tensor(val_X) == "40a61fb08e769b0ce228415d69b846826f4faf92f758dfff8710305c47662139"
assert hash_tensor(test_X) == "1f6cd72379d003c512c5ec690bae4f119d3c5f201c8f157650e531e0d2d84f7c"

In [26]:
train_y = torch.tensor(train_y.values, dtype=torch.long)
val_y   = torch.tensor(val_y.values, dtype=torch.long)
test_y  = torch.tensor(test_y.values, dtype=torch.long)

In [27]:
def create_datasets(train_X, train_y, val_X, val_y, test_X, test_y):
    """
    Creates PyTorch TensorDataset objects for the training, validation,
    and test splits.

    Parameters:
        train_X (torch.Tensor): Training input features 
        train_y (torch.Tensor): Training target labels 
        val_X (torch.Tensor): Validation input features 
        val_y (torch.Tensor): Validation target labels 
        test_X (torch.Tensor): Test input features 
        test_y (torch.Tensor): Test target labels 
        
    Returns:
        train_dataset (TensorDataset): Dataset containing training inputs and targets.
        val_dataset (TensorDataset): Dataset containing validation inputs and targets.
        test_dataset (TensorDataset): Dataset containing test inputs and targets.
    """

    ### BEGIN SOLUTION
    train_dataset = TensorDataset(train_X, train_y)
    val_dataset   = TensorDataset(val_X, val_y)
    test_dataset  = TensorDataset(test_X, test_y)

    return train_dataset, val_dataset, test_dataset
    ### END SOLUTION

In [28]:
train_dataset, val_dataset, test_dataset = create_datasets(train_X, train_y, val_X, val_y, test_X, test_y)

In [29]:
##TEST
assert isinstance(train_dataset, TensorDataset)
assert isinstance(val_dataset, TensorDataset)
assert isinstance(test_dataset, TensorDataset)
assert len(train_dataset) == train_X.shape[0] == train_y.shape[0]
assert len(val_dataset) == val_X.shape[0] == val_y.shape[0]
assert len(test_dataset) == test_X.shape[0] == test_y.shape[0]

In [30]:
### AUTOMATICALLY GRADED TASK
def create_dataloaders(train_dataset, val_dataset, test_dataset):
    """
    Constructs PyTorch DataLoader objects for the training, validation,
    and test datasets with batch size 16.

    Parameters:
        train_dataset (TensorDataset): Dataset containing training samples.
        val_dataset (TensorDataset): Dataset containing validation samples.
        test_dataset (TensorDataset): Dataset containing test samples.

    Returns:
        train_loader (DataLoader): DataLoader that yields mini-batches from the training dataset.
        val_loader (DataLoader): DataLoader that yields mini-batches from the validation dataset.
        test_loader (DataLoader): DataLoader that yields mini-batches from the test dataset.
    """
    
    ### BEGIN SOLUTION
    train_loader = DataLoader(train_dataset, batch_size=16)
    val_loader = DataLoader(val_dataset, batch_size=16)
    test_loader = DataLoader(test_dataset, batch_size=16)
    
    return train_loader, val_loader, test_loader
    ### END SOLUTION

In [31]:
train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset)

In [32]:
##TEST
assert isinstance(train_loader, DataLoader)
assert isinstance(val_loader, DataLoader)
assert isinstance(test_loader, DataLoader)
xb, yb = next(iter(train_loader))
assert xb.shape[0] <= 32   
assert yb.shape[0] == xb.shape[0]

In [33]:
### AUTOGRADED TASK
class TextLSTM(nn.Module):
    """
    LSTM-based model for text classification.

    The architecture consists of the following components:
    1. embedding: An embedding layer that maps token indices to 128-dimensional vectors (use padding_idx=0).
    2. dropout1: A dropout layer with a rate of 30% for regularization.
    3. lstm: An LSTM layer with 64 hidden units to model sequential dependencies in the input text.
    4. dropout2: A dropout layer with a rate of 30% for regularization.
    5. fc: A final fully connected linear layer that produces the output logits.

    Note:
    No activation function is applied in the forward pass. The appropriate
    activation is applied implicitly by the loss function during training.
    """
    
    ### BEGIN SOLUTION
    def __init__(self, vocab_size: int):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, 128, padding_idx=0)
        self.dropout1 = nn.Dropout(0.30)
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)
        self.dropout2 = nn.Dropout(0.30)
        self.fc = nn.Linear(64, 3) 

    def forward(self, x):
        x = self.embedding(x)      
        x = self.dropout1(x)                
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.dropout2(x)                
        return self.fc(x).squeeze(1)                
    ### END SOLUTION

In [34]:
### AUTOGRADED TASK
def initialize_model():
    """
    Initializes and returns a TextLSTM model.

    Parameters:
        vocab_size (int): Size of the vocabulary

    Returns:
        torch.nn.Module
    """

    ### BEGIN SOLUTION
    return TextLSTM(len(vocab))
    ### END SOLUTION

In [35]:
model = initialize_model()

In [36]:
##TEST
assert hasattr(model, "embedding")
assert model.embedding.embedding_dim == 128
assert hash_number(model.embedding.num_embeddings) == "782c60cb3f190b381e1bed1b8ee205b00d59fc1241c28ed88b169d092be94892"

In [37]:
##TEST
assert hasattr(model, "dropout1") 
assert model.dropout1.p == 0.3

In [38]:
##TEST
assert hasattr(model, "lstm")
assert model.lstm.input_size == 128
assert model.lstm.hidden_size == 64

In [39]:
###TEST
assert hasattr(model, "dropout2") 
assert model.dropout2.p == 0.3

In [40]:
##TEST
assert hasattr(model, "fc") 
assert model.fc.in_features == 64
assert hash_number(model.fc.out_features) == "4e07408562bedb8b60ce05c1decfe3ad16b72230967de01f640b7e4729b49fce"

In [41]:
### AUTOGRADED TASK
def get_loss_fn():
    """
    Define the loss function.

    Hint:
    Since no activation function is defined at the output layer, you should choose
    a loss function suitable for this task that internally applies the appropriate
    activation.

    Returns:
        loss_fn
    """
        
    ### BEGIN SOLUTION
    return nn.CrossEntropyLoss()
    ### END SOLUTION

In [42]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = get_loss_fn()

In [43]:
##TEST
assert hash_loss_fn(loss_fn) == "26e028292be7fff8f121074eb70733520c493600aff2bf5989b7c13a840ce9d2"

In [44]:
def train(model, dataloader, loss_fn, optimizer):
    model.train()

    total_loss = 0
    for features, targets in dataloader:
        optimizer.zero_grad()

        outputs = model(features)
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss

In [45]:
### AUTOGRADED TASK
def evaluate(model, dataloader, loss_fn):
    """
    Transform the logits in order to get the class prediction.
    """
 
    model.eval()

    total_loss, correct, total = (0,)*3
    with torch.no_grad():
        for features, targets in dataloader:
            outputs = model(features)
            loss = loss_fn(outputs, targets)
            
            ### BEGIN SOLUTION
            predictions = torch.argmax(outputs, dim=1)           
            ### END SOLUTION

            correct += (predictions == targets).sum().item()
            total += targets.numel()

            total_loss += loss.item()

    accuracy = correct / total
    return total_loss, accuracy

In [46]:
def train_and_evaluate(model, train_loader, val_loader, loss_fn, optimizer, num_epochs):
    train_losses, val_losses = [], []
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, train_loader, loss_fn, optimizer)
        train_losses.append(train_loss)
        val_loss, _ = evaluate(model, val_loader, loss_fn)
        val_losses.append(val_loss)

        print(f"Epoch {epoch:3d}/{num_epochs} | train_loss={train_loss:.4f} val_loss={val_loss:.4f}")
                  
    return train_losses, val_losses

In [ ]:
num_epochs = 20

In [ ]:
train_losses, val_losses = train_and_evaluate(model, train_loader, val_loader, loss_fn, optimizer, num_epochs)

In [ ]:
sns.lineplot(x=range(1, num_epochs + 1), y=train_losses, label="train_loss")
sns.lineplot(x=range(1, num_epochs + 1), y=val_losses, label="val_loss")

In [ ]:
test_loss, test_acc = evaluate(model, test_loader, loss_fn)

In [ ]:
test_loss, test_acc

In [ ]:
##TEST
assert isinstance(test_acc, float)
assert 0.4 <= test_acc <= 1